In [70]:
import numpy as np
import netCDF4
import h5netcdf
import xarray as xr
import sys
import tensorflow as tf
from tensorflow import keras
import pickle
from os.path import join
sys.path.append('/home/samuel.varga/projects/deep_learning/')

from deep_learning.training_utils import load_rotation, convert_to_tf
from deep_learning.deep_networks import create_U_net_classifier_2D

In [73]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
from keras import backend as K
import gc
from numba import cuda
outdir='/work/samuel.varga/projects/2to6_hr_severe_wx/DEEP_LEARNING/'

In [67]:
batch_size=1024
target_column='any_severe__36km'
rotation=4
X_train, y_train, mean, variance = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__training_data'), rotation, target_column)
X_val, y_val = load_rotation(join('/work/samuel.varga/data/2to6_hr_severe_wx/DEEP_LEARNING/',f'wofs_dl_severe__2to6hr__rot_{rotation}__validation_data'), rotation, target_column)
print(np.shape(X_train))
print(np.shape(y_train))

#Convert to tf dataset
train_ds = convert_to_tf((X_train,y_train), batch_size)
val_ds = convert_to_tf((X_val,y_val), batch_size)
test_ds=None

Training path detected - loading scaling
(6410, 16, 16, 63)
(6410, 16, 16)


In [61]:
#U-net architectural parameters
conv_filters=[64,128,256,512]
conv_size=[2,1,2,1]
max_pool=[2,2,2,2]
conv_layers =[{'filters': f, 'kernel_size': (s), 'pool_size': (p), 'strides': (p)} if p > 1
                   else {'filters': f, 'kernel_size': (s), 'pool_size': None, 'strides': None}
                   for s, f, p, in zip(conv_size, conv_filters, max_pool)]
args={'lrate':1e-4, 'loss':'sparse_categorical_crossentropy','activation_conv':'relu','activation_out':'softmax',
     'p_spatial_dropout':0.1, 'filters':conv_filters, 'size':conv_size, 'pool':max_pool, 'shape':(16,16),
     'rotation':rotation,'target_column':target_column}
metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
         #tf.keras.metrics.AUC(), 
         #tf.keras.metrics.FalseNegatives(), tf.keras.metrics.FalsePositives(),]
         #tf.keras.metrics.MeanAbsoluteError(), #tf.keras.metrics.TrueNegatives(),
         #tf.keras.metrics.TruePositives()]

In [62]:
#create u-net
#Pick better loss/activation/output functions
u_net = create_U_net_classifier_2D(image_size=args['shape'], nchannels=63, n_classes=2, conv_layers=conv_layers, p_spatial_dropout=args['p_spatial_dropout'], metrics=metrics,
                           lrate=args['lrate'], loss=args['loss'], activation_conv=args['activation_conv'], activation_out=args['activation_out'],
                                  normalization=(mean, variance))

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Input (InputLayer)             [(None, 16, 16, 63)  0           []                               
                                ]                                                                 
                                                                                                  
 normalization_10 (Normalizatio  (None, 16, 16, 63)  0           ['Input[0][0]']                  
 n)                                                                                               
                                                                                                  
 Encode_0_0 (Conv2D)            (None, 16, 16, 64)   16192       ['normalization_10[0][0]']       
                                                                                           

 De_Sp_Dr_2_1 (SpatialDropout2D  (None, 4, 4, 256)   0           ['Decode_2_1[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_2_0 (Conv2D)            (None, 4, 4, 256)    262400      ['De_Sp_Dr_2_1[0][0]']           
                                                                                                  
 De_Sp_Dr_2_0 (SpatialDropout2D  (None, 4, 4, 256)   0           ['Decode_2_0[0][0]']             
 )                                                                                                
                                                                                                  
 Decode_Upsample_1 (UpSampling2  (None, 8, 8, 256)   0           ['De_Sp_Dr_2_0[0][0]']           
 D)                                                                                               
          

In [63]:
from tensorflow.keras.utils import plot_model
%matplotlib inline
#plot_model(u_net, show_shapes=True, show_layer_names=True)

In [64]:
#Callbacks
early_stopping_cb =keras.callbacks.EarlyStopping(patience=25, restore_best_weights=True,
                                                min_delta=0.001, monitor='val_loss')
tensorboard_cb= keras.callbacks.TensorBoard(log_dir=join(outdir,'logs'), histogram_freq=1)

In [65]:
#Learn the model
history = u_net.fit(train_ds, epochs=100, verbose=True, validation_data = val_ds,
    callbacks=[early_stopping_cb, tensorboard_cb])

Epoch 1/100


2023-12-19 18:22:51.066328: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_10/En_Sp_Dr_0_0/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


7/7 [==============================] - 3s 202ms/step - loss: 0.6386 - sparse_categorical_accuracy: 0.7367 - val_loss: 0.5281 - val_sparse_categorical_accuracy: 0.9400
Epoch 2/100
7/7 [==============================] - 0s 74ms/step - loss: 0.4547 - sparse_categorical_accuracy: 0.9504 - val_loss: 0.3956 - val_sparse_categorical_accuracy: 0.9418
Epoch 3/100
7/7 [==============================] - 0s 75ms/step - loss: 0.3614 - sparse_categorical_accuracy: 0.9554 - val_loss: 0.3338 - val_sparse_categorical_accuracy: 0.9418
Epoch 4/100
7/7 [==============================] - 0s 74ms/step - loss: 0.2909 - sparse_categorical_accuracy: 0.9559 - val_loss: 0.2899 - val_sparse_categorical_accuracy: 0.9418
Epoch 5/100
7/7 [==============================] - 0s 74ms/step - loss: 0.2458 - sparse_categorical_accuracy: 0.9560 - val_loss: 0.2385 - val_sparse_categorical_accuracy: 0.9418
Epoch 6/100
7/7 [==============================] - 1s 76ms/step - loss: 0.2018 - sparse_categorical_accuracy: 0.9560 - va

Epoch 47/100
7/7 [==============================] - 0s 72ms/step - loss: 0.1177 - sparse_categorical_accuracy: 0.9577 - val_loss: 0.1641 - val_sparse_categorical_accuracy: 0.9427
Epoch 48/100
7/7 [==============================] - 0s 73ms/step - loss: 0.1165 - sparse_categorical_accuracy: 0.9574 - val_loss: 0.1651 - val_sparse_categorical_accuracy: 0.9431
Epoch 49/100
7/7 [==============================] - 0s 72ms/step - loss: 0.1165 - sparse_categorical_accuracy: 0.9571 - val_loss: 0.1653 - val_sparse_categorical_accuracy: 0.9431
Epoch 50/100
7/7 [==============================] - 0s 73ms/step - loss: 0.1175 - sparse_categorical_accuracy: 0.9576 - val_loss: 0.1651 - val_sparse_categorical_accuracy: 0.9434
Epoch 51/100
7/7 [==============================] - 0s 74ms/step - loss: 0.1166 - sparse_categorical_accuracy: 0.9575 - val_loss: 0.1642 - val_sparse_categorical_accuracy: 0.9434
Epoch 52/100
7/7 [==============================] - 0s 73ms/step - loss: 0.1154 - sparse_categorical_accu

In [71]:
fbase = f"{args['target_column']}_Rot_{args['rotation']}_{args['shape'][0]}_{args['shape'][1]}_lrate_{args['lrate']}_spatial_dropout_{args['p_spatial_dropout']}_filters_{args['filters']}_size_{args['size']}_pool_{args['pool']}"
results = {}
results['args'] = args
results['predict_val'] = u_net.predict(val_ds)
results['predict_val_eval'] = u_net.evaluate(val_ds)

if test_ds is not None:
    results['predict_test']=u_net.predict(test_ds)
    results['predict_test_eval']=u_net.evaluate(test_ds)

results['predict_train']=u_net.predict(train_ds)
results['predict_train_eval']=u_net.evaluate(train_ds)
results['history']=history.history
results['fname_base']=fbase

#Save results
with open(join(join(outdir, 'results'), f'{fbase}_results.pkl'),'wb') as fp:
    pickle.dump(results, fp)

#save model
if False:
    u_net.save(join(join(outdir, 'models'), f'{fbase}_model'))
print(fbase)

7/7 [==============================] - 0s 16ms/step - loss: 0.1143 - sparse_categorical_accuracy: 0.9580
any_severe__36km_Rot_4_16_16_lrate_0.0001_spatial_dropout_0.1_filters_[64, 128, 256, 512]_size_[2, 1, 2, 1]_pool_[2, 2, 2, 2]


In [75]:
K.clear_session()
gc.collect()
#gpu = cuda.get_current_device()
#gpu.reset()
cuda.close()